In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rumitpathare/indian-recipes")

print("Path to dataset files:", path)

In [ ]:
!cp  /home/tj/.cache/kagglehub/datasets/rumitpathare/indian-recipes/versions/22/Food_Recipe.csv ./recipes_data.csv

In [ ]:
import pandas as pd
from io import StringIO
from tqdm import tqdm

def save_dataset_with_skip(data_file, output_file):
    skipped_rows = 0
    valid_rows = []

    with open(data_file, "r", encoding="utf-8") as file:
        total_lines = sum(1 for _ in file)
        file.seek(0)
        for line in tqdm(file, total=total_lines, desc="Processing lines"):
            try:
                row = pd.read_csv(StringIO(line), header=None)
                valid_rows.append(line)
            except pd.errors.ParserError:
                skipped_rows += 1

    print(f"Skipped rows: {skipped_rows}")

    with open(output_file, "w", encoding="utf-8") as out_file:
        out_file.writelines(valid_rows)

data_file = './recipes_data.csv'
output_file = './pure_recipes_data.csv'
save_dataset_with_skip(data_file, output_file)
print(f"Filtered dataset saved to {output_file}")

In [16]:
import pandas as pd
import json
import regex as re

# 데이터 로드
df = pd.read_csv('pure_recipes_data.csv')

# 컬럼명 정리
df.rename(columns={
    'prep_time (in mins)': 'prep_time_in_mins',
    'cook_time (in mins)': 'cook_time_in_mins',
}, inplace=True)

# 결측값 채우기
df['ingredients_name'] = df['ingredients_name'].fillna('')
df['ingredients_quantity'] = df['ingredients_quantity'].fillna('')

# 정규표현식 패턴 정의 (영어, 숫자, 특수문자만 허용)
valid_pattern = re.compile(r'^[\p{Latin}\p{Nd}\p{P}\s]*$')

# 각 행에서 모든 텍스트가 유효한지 확인하는 함수
def is_row_valid(row):
    for col in ['name', 'cuisine', 'course', 'diet', 'ingredients_name', 'ingredients_quantity', 'instructions']:
        if not re.match(valid_pattern, str(row[col])):
            return False
    return True

# 유효한 행만 필터링
df = df[df.apply(is_row_valid, axis=1)]

# instruction 생성
df['instruction'] = df['ingredients_name'].apply(
    lambda x: f'I have the following ingredients: {x}. Please provide recipes I can make with them.'
)

# 재료 및 지시사항 정리 함수
def format_output(row):
    # JSON 헤더 생성
    json_header = json.dumps({"is_recipe_request": True, "recipe_name": row['name']})
    output = f"{json_header}\n"
    output += f"Recipe Name: {row['name']}\n"
    output += f"Cuisine: {row['cuisine']}\n"
    output += f"Course: {row['course']}\n"
    output += f"Diet: {row['diet']}\n"
    output += "Ingredients:\n"

    # 재료 정리 및 번호 매기기
    ingredients = row['ingredients_name'].split(',')
    quantities = row['ingredients_quantity'].split('  ')

    cleaned_ingredients = []
    for idx, (name, qty) in enumerate(zip(ingredients, quantities), start=1):
        cleaned_name = name.strip()
        cleaned_qty = qty.strip()
        if cleaned_name and cleaned_qty:
            cleaned_ingredients.append(f"{idx}. {cleaned_name}: {cleaned_qty}")

    output += "\n".join(cleaned_ingredients) + "\n"

    # 지시사항 정리 및 번호 매기기
    output += "Instructions:\n"
    instructions = re.split(r'[.!?]\s*', row['instructions'])  # 문장별로 분리
    instructions = [instr.strip() for instr in instructions if instr.strip()]  # 빈 문장 제거

    for idx, instruction in enumerate(instructions, start=1):
        output += f"{idx}. {instruction}.\n"

    return output

# output 생성
df['output'] = df.apply(format_output, axis=1)

# input 컬럼 초기화
df['input'] = ''

# 필요한 컬럼만 선택
df = df[['instruction', 'input', 'output']]

# 결과 확인
print(df.head())

# CSV로 저장
df.to_csv('processed_dataset.csv', index=False, quotechar='"', escapechar="\\")


                                         instruction input  \
0  I have the following ingredients: Tortillas, C...         
1  I have the following ingredients: Cashew nuts,...         
2  I have the following ingredients: Bhindi (Lady...         
4  I have the following ingredients: Rice, Brinja...         
8  I have the following ingredients: Small Brinja...         

                                              output  
0  {"is_recipe_request": true, "recipe_name": "Me...  
1  {"is_recipe_request": true, "recipe_name": "Ol...  
2  {"is_recipe_request": true, "recipe_name": "An...  
4  {"is_recipe_request": true, "recipe_name": "On...  
8  {"is_recipe_request": true, "recipe_name": "Sp...  


* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/home/tj/anaconda3/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3080. Max memory: 9.753 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [vicgalle](https://huggingface.co/datasets/vicgalle/alpaca-gpt4), which is a version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html) generated from GPT4. You can replace this code section with your own data prep.

In [3]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='processed_dataset.csv', split='train')
print(dataset.column_names)

Generating train split: 6444 examples [00:00, 44345.65 examples/s]

['instruction', 'input', 'output']


One issue is this dataset has multiple columns. For `Ollama` and `llama.cpp` to function like a custom `ChatGPT` Chatbot, we must only have 2 columns - an `instruction` and an `output` column.

In [4]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
    output_column_name = "output",
    conversation_extension = 3, # Select more to handle longer conversations
)

Extending conversations: 100%|██████████| 6444/6444 [00:00<00:00, 17733.75 examples/s]


Finally use `standardize_sharegpt` to fix up the dataset!

In [5]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

Standardizing format: 100%|██████████| 6444/6444 [00:00<00:00, 17036.86 examples/s]


### Customizable Chat Templates

You also need to specify a chat template. Previously, you could use the Alpaca format as shown below.

The issue is the Alpaca format has 3 fields, whilst OpenAI style chatbots must only use 2 fields (instruction and response). That's why we used the `to_sharegpt` function to merge these columns into 1.

In [6]:
chat_template = """Below are some instructions that describe some tasks. Start your response with a JSON object, where is_recipe_request is true if the instruction is requesting a recipe (false otherwise), and recipe_name is the extracted recipe name or null if none. Follow this with a plain text answer detailing the response.

### Instruction:
{INPUT}

### Response:
{OUTPUT}
"""



from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    default_system_message = "Start your response with a JSON object including is_recipe_request and recipe_name. is_recipe_request should be true if the instruction requests a recipe, false otherwise. recipe_name should be the name of the recipe requested or null if none. Follow with a plain text answer detailing the response."
)


Unsloth: We automatically added an EOS token to stop endless generations.
Map: 100%|██████████| 6444/6444 [00:00<00:00, 10849.61 examples/s]


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 11,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 60,
        num_train_epochs = 1, # For longer training runs!
        learning_rate = 1e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=11): 100%|██████████| 6444/6444 [00:03<00:00, 1660.13 examples/s]


In [8]:
trainer_stats = trainer.train()  

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,444 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 805
 "-____-"     Number of trainable parameters = 41,943,040
  0%|          | 1/805 [00:10<2:19:43, 10.43s/it]

{'loss': 1.4383, 'grad_norm': 0.3933122158050537, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


  0%|          | 2/805 [00:19<2:08:29,  9.60s/it]

{'loss': 1.5581, 'grad_norm': 0.3952540159225464, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}


  0%|          | 3/805 [00:27<2:01:02,  9.06s/it]

...

{'loss': 0.8106, 'grad_norm': 0.5236582159996033, 'learning_rate': 3.855309264721996e-11, 'epoch': 1.0}


100%|██████████| 805/805 [1:59:43<00:00,  9.58s/it]

{'loss': 0.725, 'grad_norm': 0.4440672695636749, 'learning_rate': 0.0, 'epoch': 1.0}


100%|██████████| 805/805 [1:59:44<00:00,  8.92s/it]

{'train_runtime': 7184.4191, 'train_samples_per_second': 0.897, 'train_steps_per_second': 0.112, 'train_loss': 0.8360126417616139, 'epoch': 1.0}


<a name="Inference"></a>
### Inference
Let's run the model! Unsloth makes inference natively 2x faster as well! You should use prompts which are similar to the ones you had finetuned on, otherwise you might get bad results!

In [9]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "I have beef and salt and rice, What can i make?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# _ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)
_ = model.generate(input_ids, streamer=text_streamer, max_new_tokens=128, attention_mask=input_ids.ne(tokenizer.pad_token_id).to("cuda"), pad_token_id=tokenizer.pad_token_id)

{"is_recipe_request": true, "recipe_name": "Beef and Rice Recipe - Beef Biryani Recipe"}
Recipe Name: Beef and Rice Recipe - Beef Biryani Recipe
Cuisine: Indian
Course: Lunch
Diet: Non Vegeterian
Ingredients:
1. Beef: 500 grams Beef, cut into small pieces
2. Salt, to taste
3. Rice: 1 cup Rice
4. Oil, for cooking
5. Onions, chopped
6. Ginger, chopped
7. Garlic, chopped
8. Cinnamon Stick
9. Cloves
10. Cardam


In [10]:
if True: model.save_pretrained_gguf("taejoonlee/v7", tokenizer, maximum_memory_usage=0.8)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 15.32 out of 31.26 RAM for saving.


100%|██████████| 32/32 [00:37<00:00,  1.16s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at taejoonlee/v7 into q8_0 GGUF format.
The output location will be ./taejoonlee/v7/unsloth.Q8_0.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: v7
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> Q8_0, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      t

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: ./taejoonlee/v7/unsloth.Q8_0.gguf
Unsloth: Saved Ollama Modelfile to taejoonlee/v7/Modelfile
